# Data cleaning

This notebook's goal is to clean the dataset based on the conclusions of the univariate eda analysis.
The bivariate analysis will focus first on Production, Source and Seasons.

> Extra: If time is not limited we will also work with the month aggregation of the production values with sources Wind and Solar.

In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from IPython.display import clear_output

# Filepath to the dataset
file_path = "Energy Production Dataset.csv"

# Load the latest version
data = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "ahmeduzaki/wind-and-solar-energy-production-dataset",
  file_path,
)

clear_output()
print("Dataset loaded successfully!")


Dataset loaded successfully!


## Removing the Mixed source.

In [2]:
data.Source.unique()

<StringArray>
['Wind', 'Solar', 'Mixed']
Length: 3, dtype: str

In [3]:
data.shape

(51864, 9)

In [4]:
# Filter out mixed source

data = data[data.Source != "Mixed"]
data.shape

(51862, 9)

## Remove Start_Hour and End_Hour 

We are focus our efforts on the month aggregation for now, so these columns can be dropped.

In [5]:
columns = data.columns.tolist()
columns = [ item for item in columns if item not in ['Start_Hour', 'End_Hour', 'Day_of_Year', 'Day_Name']]

print(columns)

['Date', 'Source', 'Month_Name', 'Season', 'Production']


## Aggregating the month

In [6]:
data = data[columns]
data.head()

,Date,Source,Month_Name,Season,Production
0,11/30/2025,Wind,November,Fall,5281
1,11/30/2025,Wind,November,Fall,3824
2,11/30/2025,Wind,November,Fall,3824
3,11/30/2025,Wind,November,Fall,6120
4,11/30/2025,Wind,November,Fall,4387


In [7]:
import pandas as pd
data.Date = pd.to_datetime(data.Date)
data["Year"] = data.Date.apply(lambda x: x.year)
data["Month"] = data.Date.apply(lambda x: x.month)


In [8]:
data.head()

,Date,Source,Month_Name,Season,Production,Year,Month
0,2025-11-30,Wind,November,Fall,5281,2025,11
1,2025-11-30,Wind,November,Fall,3824,2025,11
2,2025-11-30,Wind,November,Fall,3824,2025,11
3,2025-11-30,Wind,November,Fall,6120,2025,11
4,2025-11-30,Wind,November,Fall,4387,2025,11


In [9]:
columns = [ item for item in data.columns if item !="Date"]
data = data[columns]
data.head()

,Source,Month_Name,Season,Production,Year,Month
0,Wind,November,Fall,5281,2025,11
1,Wind,November,Fall,3824,2025,11
2,Wind,November,Fall,3824,2025,11
3,Wind,November,Fall,6120,2025,11
4,Wind,November,Fall,4387,2025,11


In [10]:
grouped = data.groupby(["Year", "Month", "Source"]).agg({"Production": "sum", "Month_Name": "first", "Season": "first"})

In [11]:
grouped = grouped.reset_index()
grouped[grouped.Year == 2024]


,Year,Month,Source,Production,Month_Name,Season
95,2024,1,Solar,224043,January,Winter
96,2024,1,Wind,5699660,January,Winter
97,2024,2,Solar,129640,February,Winter
98,2024,2,Wind,6266479,February,Winter
99,2024,3,Solar,713437,March,Spring
100,2024,3,Wind,4927937,March,Spring
101,2024,4,Solar,706113,April,Spring
102,2024,4,Wind,5015455,April,Spring
103,2024,5,Solar,1190756,May,Spring
104,2024,5,Wind,2653225,May,Spring


In [12]:
# To work with unaggregated by month, only cleaned
csv_filename = "cleaned-wind-and-solar-energy.csv"

data.to_csv(csv_filename)

In [13]:
# To work with month aggregation
csv_filename = "month_aggregated-wind-and-solar-energy.csv"

grouped.to_csv(csv_filename)